In [1]:
#basic
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

#dataset/dataloader stuff
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image
import torchvision.transforms as transforms 
from torchvision.io import ImageReadMode

#network stuff
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
#pointless now
#datasets dont need gpus to function but usually on top
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
RANDOM_SEED = 42
DEVICE

'cpu'

In [3]:
class LeBirbs(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.image_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_labels.iloc[idx]['image_path']
        img_path = img_path[1:]
        img_path = os.path.join(self.image_dir,img_path)
        image = read_image(img_path, mode = ImageReadMode.RGB)
        image = image.to(torch.float)
        label = self.img_labels.iloc[idx]['label']
        label = torch.tensor(label)
        
        if self.transform:
            transform = transforms.transforms.CenterCrop(250)
            image = transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [5]:
#initiate the dataset object
train_dataset = LeBirbs('train_images.csv','train_images',True,None)
test_dataset = LeBirbs('test_images_path.csv','test_images',True,None)
train_dataloader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=64,shuffle=False)

In [7]:
#implementation without atributes
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(55696,7000)
        self.fc2 = nn.Linear(7000, 1000)
        self.fc3 = nn.Linear(1000, 201)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
num_epochs = 25

In [11]:
%%time
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Finished Training
CPU times: total: 23h 6min 57s
Wall time: 5h 47min 17s


In [ ]:
#can save trained models, too large file size for github
#PATH = 'Nets/DK_1_net.pth'
#torch.save(net.state_dict(), PATH)

In [12]:
test_file = pd.read_csv('test_images_sample.csv')

all_predictions = []
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions = torch.Tensor.tolist(predicted)
        all_predictions.extend(predictions)

In [13]:
test_file['label'] = all_predictions
test_file.to_csv('test_images_predictions_dk.csv',index=False)